<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/MENA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import necessary modules
import csv
with open('2015.csv','rt')as f:
  data = csv.reader(f)


In [0]:
import pandas as pd
data = pd.read_csv('TDS2017.csv',encoding = "ISO-8859-1", engine = 'python')
